# Запитання: Як варіюється ціна за експортований кілограм в залежності від регіону?

In [1]:
import altair as alt
import pandas as pd
import altair_viewer
import warnings
warnings.filterwarnings('ignore')
alt.renderers.enable('altair_viewer')

RendererRegistry.enable('altair_viewer')

In [2]:
df = pd.read_csv('gapminder_country_data.csv')
df.head()

,country,time,income_per_person_gdppercapita_ppp_inflation_adjusted,life_expectancy_years,population_total,name,world_4region
0,afg,2019,1763.0,64.08,38041757,Afghanistan,Asia
1,ago,2019,5544.0,65.00,31825299,Angola,Africa
2,alb,2019,12694.0,78.47,2880913,Albania,Europe
3,and,2019,53245.0,NaN,77146,Andorra,Europe
4,are,2019,65650.0,73.63,9770526,United Arab Emirates,Asia


In [3]:
df.drop(['time'], axis = 1, inplace = True)
df.columns = ['id', 'gdp_per_cap', 'life_expectancy', 'population', 'country', 'region']
df.drop(['gdp_per_cap'], axis = 1, inplace = True)
df.drop(['life_expectancy'], axis = 1, inplace = True)
df.drop(['population'], axis = 1, inplace = True)
df.id = df.id.str.upper()
df.head()

,id,country,region
0,AFG,Afghanistan,Asia
1,AGO,Angola,Africa
2,ALB,Albania,Europe
3,AND,Andorra,Europe
4,ARE,United Arab Emirates,Asia


In [4]:
data = pd.read_csv('UNdata_Export_20211219_151223778.csv')

data['country'] = data['Country or Area']

In [5]:
data.head()

,Country or Area,Year,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity,country
0,Afghanistan,2018,"Cocoa powder, unsweetened",Import,7.247329e+04,8.070000e+04,Weight in kilograms,8.070000e+04,Afghanistan
1,Afghanistan,2018,Chocolate/cocoa food preparations nes,Import,3.281740e+07,5.662823e+06,Weight in kilograms,5.662823e+06,Afghanistan
2,Afghanistan,2018,Chocolate/cocoa food preparations nes,Export,4.327404e+06,0.000000e+00,Weight in kilograms,6.239175e+06,Afghanistan
3,Afghanistan,2018,Chocolate/cocoa food preparations nes,Re-Export,7.452492e+04,0.000000e+00,Weight in kilograms,0.000000e+00,Afghanistan
4,Afghanistan,2016,Chocolate/cocoa food preparations nes,Import,1.568728e+08,2.677301e+07,Weight in kilograms,2.677301e+07,Afghanistan


In [6]:
data.shape

(65353, 9)

In [7]:
to_drop = [i for i, r in data[data['Weight (kg)']==0.0].iterrows()]
data.drop(to_drop, axis = 0, inplace = True)
data.shape

(64886, 9)

In [8]:
data['$/kg'] = data['Trade (USD)']/data['Weight (kg)']

In [9]:
data= data[data['Flow']=='Export']
data.shape

(24116, 10)

In [10]:
data.head()

,Country or Area,Year,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity,country,$/kg
16,Albania,2018,Chocolate/cocoa food preparations nes,Export,28830.057548,12523.0,Weight in kilograms,12523.0,Albania,2.302169
23,Albania,2017,Chocolate/cocoa food preparations nes,Export,113662.252266,49663.2,Weight in kilograms,49663.2,Albania,2.288661
29,Albania,2016,"Chocolate, cocoa prep, block/slab/bar, not fil...",Export,2302.000000,640.0,Weight in kilograms,640.0,Albania,3.596875
31,Albania,2016,Chocolate/cocoa food preparations nes,Export,105124.000000,41833.0,Weight in kilograms,41833.0,Albania,2.512944
38,Albania,2015,Chocolate/cocoa food preparations nes,Export,102143.000000,50167.0,Weight in kilograms,50167.0,Albania,2.036060


In [11]:
data.groupby(['country' ]).sum().reset_index()
data.shape

(24116, 10)

In [12]:
data.head()

,Country or Area,Year,Commodity,Flow,Trade (USD),Weight (kg),Quantity Name,Quantity,country,$/kg
16,Albania,2018,Chocolate/cocoa food preparations nes,Export,28830.057548,12523.0,Weight in kilograms,12523.0,Albania,2.302169
23,Albania,2017,Chocolate/cocoa food preparations nes,Export,113662.252266,49663.2,Weight in kilograms,49663.2,Albania,2.288661
29,Albania,2016,"Chocolate, cocoa prep, block/slab/bar, not fil...",Export,2302.000000,640.0,Weight in kilograms,640.0,Albania,3.596875
31,Albania,2016,Chocolate/cocoa food preparations nes,Export,105124.000000,41833.0,Weight in kilograms,41833.0,Albania,2.512944
38,Albania,2015,Chocolate/cocoa food preparations nes,Export,102143.000000,50167.0,Weight in kilograms,50167.0,Albania,2.036060


In [13]:
n = data.iloc[:,-2:]
n.head()

,country,$/kg
16,Albania,2.302169
23,Albania,2.288661
29,Albania,3.596875
31,Albania,2.512944
38,Albania,2.036060


In [14]:
n = n.groupby(['country' ]).sum().reset_index()

In [15]:
merged = n.merge(df)
merged.head()

,country,$/kg,id,region
0,Albania,124.697525,ALB,Europe
1,Algeria,1389.680428,DZA,Africa
2,Andorra,298.298732,AND,Europe
3,Angola,75.029208,AGO,Africa
4,Antigua and Barbuda,462.859807,ATG,Americas


In [16]:
data['year'] = data['Year'].astype(str)
options=[None]
options.extend(sorted(merged.region.unique().tolist()))
labels=['All']
labels.extend(sorted(merged.region.unique().tolist()))

input_dropdown = alt.binding_select(options = options, labels=labels)
select_region = alt.selection_single(fields = ['region'], bind = input_dropdown, name='Choose')

In [17]:
regions = labels

In [18]:
#color changing marks
histogram_context  = alt.Chart(merged).mark_bar(color = 'lightgray').encode(
    alt.X("$/kg:Q", bin=alt.Bin(extent=[0, 5000], step=200), title='Price per kg of goods, $'),
    y='count()',
)

hist = alt.Chart(merged).mark_bar().encode(
    alt.X("$/kg:Q", bin=alt.Bin(extent=[0, 5000], step=200)),
    y='count()',
)



rating_radio = alt.binding_radio(options = options, labels=labels)

rating_select = alt.selection_single(fields=['region'], bind=rating_radio, name="Select")
rating_color_condition = alt.condition(rating_select,
#                         alt.Color('region:N'),
                      alt.value('#3a9aba'),
                      alt.value('lightgray'))

hist = hist.add_selection(
    rating_select
).transform_filter(
    rating_select
).encode(
    color=alt.value('#3a9aba')
).properties(title="Price per kilogram of produced goods")

(histogram_context + hist).properties(width = 800, height = 400).configure_title(fontSize=25) .configure_axis(
    labelFontSize=10,
    titleFontSize=17
)

Displaying chart at http://localhost:23113/

Наступні візуалізації -- це просто спроби знайти більш ефективне рішення

In [42]:
colls = [7,8,9]
n2 = data.iloc[:,colls]
n2 = n2.groupby(['country',]).sum().reset_index()
merged2 = n2.merge(df)
merged2.head()

,country,Quantity,$/kg,id,region
0,Albania,614087.20,124.697525,ALB,Europe
1,Algeria,22945574.00,1389.680428,DZA,Africa
2,Andorra,7703481.72,298.298732,AND,Europe
3,Angola,56236.19,75.029208,AGO,Africa
4,Antigua and Barbuda,35931.51,462.859807,ATG,Americas


In [43]:
merged2.shape

(146, 5)

In [49]:
merged2=merged2[merged2['$/kg']<20000]
merged2.shape

(145, 5)

In [51]:
scatterplot = alt.Chart(merged2).mark_point(filled = True).encode(
    x = alt.X('Quantity:Q',
             
#              scale = alt.Scale(type = 'log', base = 2)
             ),
    y = alt.Y('$/kg:Q',
#               scale = alt.Scale(range = [0, 2000])
             ),
    color = alt.Color('region:N'),
    size = alt.Size('$/kg:Q'),
    tooltip = alt.Tooltip('country:N')
)
scatterplot


Displaying chart at http://localhost:16234/